## Overview
This notebook creates an initial dataframe of unique patient admissions and ICU stays alongside key demographics.
Makes use of 3 MIMIC-III tables: Patients, Admissions, and ICUStays.

## Notebook sections
#### a) Library Import and BigQuery Auth Setup for utilizing SQL
#### b) Data Cleaning Key Activities:
    1) Ethnicities mapped to smaller groups in a new columns
    2) Ages > 300 represent patients aver age 89 and are recoded as age 90
#### c) Labeling of patients for potential exclusion:
    1) lack of ICU stay
    2) young age and therefore minimal likelihood of acquiring C Diff infection
    3) Pre-identified C. Diff or diarrhea upon admission to hospital

## Output
#### Two files:
    1) Patient Admits dataframe
    2) ICU Stays dataframe     

## A) Import libraries and set up BigQuery authorization

In [1]:
#Import google bigquery and authorization library, and set up client and project name for billing
from google.cloud import bigquery
from google_auth_oauthlib import flow

client = bigquery.Client()

project = 'mimiciii-249117'

In [2]:
# Import libraries for analysis within notebook
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Next three cells launch browser and run through end-user login for access to mimic-iii project hosted on BigQuery 
launch_browser = True

In [4]:
appflow = flow.InstalledAppFlow.from_client_secrets_file(
    'C:/Users/fuzzyman27/BigQuery/client_secrets.json',
    scopes=['https://www.googleapis.com/auth/bigquery'])

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()

credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=579414793473-ak30u31klvdecqfv6kg8542ofeeegm18.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=8RExhDaHPAUUCyHNTgIiZ7AamioQiB&access_type=offline&code_challenge=FQrNPym0fPLjxzbVm8O8EJ4WCzlcUErLW2TUp28s-2c&code_challenge_method=S256


In [5]:
client = bigquery.Client(project=project, credentials=credentials)

In [6]:
# Define small function to run SQL queries and return a dataframe
def run_query(query):
    return client.query(query).to_dataframe()

## B) Data Cleaning

### Patient and Admissions Data
Leverages both patients and admissions tables

#### Load Data

In [32]:
# Combines patients and admissions table and extract all fields
# Three calculated fields include length of stay, patient age, and sequence of hospital stay
# for each hospital admission, and sequence of hospital admission for each patientCode from here in part inspired by calculated fields here

# https://github.com/MIT-LCP/mimic-code/blob/master/concepts/demographics/icustay-detail.sql
query = """
    SELECT p.subject_id, p.gender, p.dob, p.dod, p.dod_hosp, p.dod_ssn,
    p.expire_flag, a.hadm_id, a.admittime, a.dischtime, a.deathtime,
    a.admission_type, a.admission_location, a.discharge_location,
    a.insurance, a.language, a.religion, a.marital_status, a.ethnicity,
    a.edregtime, a.edouttime, a.diagnosis, a.has_chartevents_data,
    datetime_diff(a.dischtime, a.admittime, second) / (60*60*24) as hosp_los,
    datetime_diff(a.admittime, p.dob, second) / (60*60*24*365.242) as admit_age,
    DENSE_RANK() OVER (PARTITION BY a.subject_id ORDER BY a.admittime) as hospstay_seq
    FROM `physionet-data.mimiciii_clinical.admissions` as a
    JOIN `physionet-data.mimiciii_clinical.patients` as p
    on (a.subject_id = p.subject_id)
    ;
"""
patient_admits = run_query(query)

#### Initial data cleaning

In [33]:
# Check data types
patient_admits.dtypes

subject_id                       int64
gender                          object
dob                     datetime64[ns]
dod                     datetime64[ns]
dod_hosp                datetime64[ns]
dod_ssn                 datetime64[ns]
expire_flag                      int64
hadm_id                          int64
admittime               datetime64[ns]
dischtime               datetime64[ns]
deathtime               datetime64[ns]
admission_type                  object
admission_location              object
discharge_location              object
insurance                       object
language                        object
religion                        object
marital_status                  object
ethnicity                       object
edregtime               datetime64[ns]
edouttime               datetime64[ns]
diagnosis                       object
has_chartevents_data             int64
hosp_los                       float64
admit_age                      float64
hospstay_seq             

In [34]:
# Check to confirm gender has two possible values and
# change gender to binary feature with 1 == F and 0 == M
patient_admits["gender"].value_counts()

M    32950
F    26026
Name: gender, dtype: int64

In [35]:
patient_admits["gender"] = patient_admits["gender"].map(lambda x: 1 if x == "F" else 0)

#### Check for missing values and adjust accordingly

In [36]:
patient_admits.isnull().sum()

subject_id                  0
gender                      0
dob                         0
dod                     36390
dod_hosp                43905
dod_ssn                 39907
expire_flag                 0
hadm_id                     0
admittime                   0
dischtime                   0
deathtime               53122
admission_type              0
admission_location          0
discharge_location          0
insurance                   0
language                25332
religion                  458
marital_status          10128
ethnicity                   0
edregtime               28099
edouttime               28099
diagnosis                  25
has_chartevents_data        0
hosp_los                    0
admit_age                   0
hospstay_seq                0
dtype: int64

##### Time of death missing values
There are 4 main columns relating to time of death of patients.

Given feature engineering will include calculating difference between time of death and other features, null values for patients still living are kept blank so as not to cause potential confusion in calculated results.

Outline of column definitions:

    1) DOD - overall time of death for patients
    2) DOD_Hosp as recorded in hospital database
    3) DOD_SSN = date of death from social 
    4) deathtime - time of death as recorded for specific hospital admission

##### Emergency room in and out time missing values

Similar to time of death, these values are stored as date times and are null when the characteristic is note relevant (eg patient did not utilize emergency room). To avoid potential calculation conflicts when comparing dates, these values are left as nulls

##### Language, Religion, and Marital Status Missing Values

Fill missing values with "None".

If found to be an importance factor in likelihood of CDI, data imputation techniques will be considered.

In [37]:
patient_admits["language"].fillna("None", inplace=True)

In [38]:
patient_admits["religion"].fillna(value="None", inplace=True)

In [39]:
patient_admits["marital_status"].fillna(value="None", inplace=True)

##### Diagnosis Missing Values

In [40]:
# Fill nulls for diagnosis column to enable calculations and convert to lowercase strings
patient_admits["diagnosis"].fillna(value = "", inplace=True)
patient_admits["diagnosis"] = patient_admits["diagnosis"].str.lower()

#### Recategorize ethnicity

Many patients are listed with two part ethnicity. Splitting these values below enables consolidating into fewer groups for analysis with larger sample sizes

In [41]:
patient_admits["ethnicity"].value_counts()

WHITE                                                       40996
BLACK/AFRICAN AMERICAN                                       5440
UNKNOWN/NOT SPECIFIED                                        4523
HISPANIC OR LATINO                                           1696
OTHER                                                        1512
ASIAN                                                        1509
UNABLE TO OBTAIN                                              814
PATIENT DECLINED TO ANSWER                                    559
ASIAN - CHINESE                                               277
HISPANIC/LATINO - PUERTO RICAN                                232
BLACK/CAPE VERDEAN                                            200
WHITE - RUSSIAN                                               164
MULTI RACE ETHNICITY                                          130
BLACK/HAITIAN                                                 101
ASIAN - ASIAN INDIAN                                           85
WHITE - OT

In [42]:
split_ethnicity = patient_admits["ethnicity"].str.split("-")
first_ethnicity = split_ethnicity.map(lambda x: x[0].strip())
num_ethnicities = split_ethnicity.map(lambda x: len(x))
second_ethnicity = split_ethnicity.map(lambda x: x[1].strip() if len(x) == 2 else "N/A")

In [43]:
first_ethnicity.value_counts()

WHITE                                                       41325
BLACK/AFRICAN AMERICAN                                       5440
UNKNOWN/NOT SPECIFIED                                        4523
ASIAN                                                        2007
HISPANIC OR LATINO                                           1696
OTHER                                                        1512
UNABLE TO OBTAIN                                              814
PATIENT DECLINED TO ANSWER                                    559
HISPANIC/LATINO                                               432
BLACK/CAPE VERDEAN                                            200
MULTI RACE ETHNICITY                                          130
BLACK/HAITIAN                                                 101
PORTUGUESE                                                     61
AMERICAN INDIAN/ALASKA NATIVE                                  51
BLACK/AFRICAN                                                  44
MIDDLE EAS

In [44]:
patient_admits["first_ethnicity"] = first_ethnicity

In [45]:
second_ethnicity.value_counts()

N/A                         57717
CHINESE                       277
PUERTO RICAN                  232
RUSSIAN                       164
ASIAN INDIAN                   85
OTHER EUROPEAN                 81
DOMINICAN                      78
BRAZILIAN                      59
VIETNAMESE                     53
GUATEMALAN                     40
FILIPINO                       25
EASTERN EUROPEAN               25
CUBAN                          24
SALVADORAN                     19
CAMBODIAN                      17
OTHER                          17
CENTRAL AMERICAN (OTHER)       13
KOREAN                         13
MEXICAN                        13
COLOMBIAN                       9
JAPANESE                        7
THAI                            4
HONDURAN                        4
Name: ethnicity, dtype: int64

In [46]:
patient_admits["second_ethnicity"] = second_ethnicity

#### Adjust Age

Ages above 89 are automatically shifted to age 300+ to protect patient privacy. Below cells recode age as 90. Given the goal of maintaining patient privacy, age imputation for these patients is not considered here.

In [47]:
patient_admits["admit_age"] = patient_admits["admit_age"].map(lambda x: x if x < 300 else 90.0)

### ICU Stays Data
Leverages icustays table

#### Load Data

In [ ]:
# Select key ICU metrics including calculated sequence of ICU stay
query = """
    SELECT subject_id, hadm_id, icustay_id, intime, outtime, los,
    DENSE_RANK() OVER (PARTITION BY hadm_id, subject_id ORDER BY intime) as icu_stay_seq
    FROM `physionet-data.mimiciii_clinical.icustays`
    ;
"""
icu_stays = run_query(query)

In [ ]:
# Calculate LOS metics across all ICU stays for a given hospital admission
query = """
    SELECT icu.hadm_id as hadm_id, sum(icu.LOS) as admit_icu_los,
    count(icu.icustay_id) as admit_icu_stays
    FROM `physionet-data.mimiciii_clinical.icustays` as icu
    GROUP BY icu.hadm_id
    ;
"""
icu_stats_by_admit = run_query(query)

#### Data Cleaning

In [ ]:
icu_stays.isnull().sum()

In [ ]:
icu_stats_by_admit.isnull().sum()

there are 10 icu stays for which there is no recorded out time and therefore no LOS

In [ ]:
# Determine which subjects and hospital admissions are associated with a
# missing ICU LOS
icu_stays[icu_stays["los"].isnull()]

In [ ]:
# Create list of hadm id's associated with missing icu LOS
missing_icu_los_hadm = icu_stays[icu_stays["los"].isnull()]["hadm_id"].to_list()

In [ ]:
# Filter ICU stays by hadm id's
icu_stays[icu_stays["hadm_id"].isin(missing_icu_los_hadm)]

In [ ]:
# Filter ICU stats hadm id's
icu_stats_by_admit[icu_stats_by_admit["hadm_id"].isin(missing_icu_los_hadm)]

In [ ]:
# Filter patient_admits by hadm id's
patient_admits[(patient_admits["hadm_id"].isin(missing_icu_los_hadm))][["subject_id",
                                                                      "dod",
                                                                      "dod_hosp",
                                                                      "dod_ssn",
                                                                      "expire_flag",
                                                                      "hadm_id",
                                                                      "admission_location",
                                                                      "discharge_location",
                                                                      "hosp_los",
                                                                      "admittime",
                                                                      "dischtime",
                                                                      "edregtime",
                                                                      "edouttime"]]

all 10 patients with missing ICU outtimes do have hospital discharge times and were only in 1 unique ICU stay over the course of the hospital visit. As such, will label ICU outtime and LOS with overall hospital discharge time

In [ ]:
for icu_index in icu_stays[icu_stays["hadm_id"].isin(missing_icu_los_hadm)].index:
    hadm_id = icu_stays.loc[icu_index, "hadm_id"]
    icu_stats_index = icu_stats_by_admit[icu_stats_by_admit["hadm_id"] == hadm_id].index[0]
    outtime = patient_admits[patient_admits["hadm_id"] == hadm_id]["dischtime"].to_list()[0]
    icu_stays.loc[icu_index, "outtime"] = outtime
    intime = icu_stays.loc[icu_index, "intime"]
    los = (outtime - np.timedelta64(0,'s')) - ((intime - np.timedelta64(0,'s')))
    icu_stats_by_admit.loc[icu_stats_index,"admit_icu_los"]=los
    icu_stays.loc[icu_index, "los"] = los

#### Merge ICU Stats and Patient Admits

In [ ]:
patient_admits = pd.merge(patient_admits, icu_stats_by_admit, how = "outer", left_on = "hadm_id", right_on = "hadm_id")

## C) Label Patient Admits For Potential Exclusion

1) Patients labeled as having CDI upon admission will not be considered for further analysis

2) Patients without an ICU stay

3) Patients with Diarrhea Upon Admission - depending on whether or not CDI testing was performed close to admission, these patients may be excluded

4) Admissions for Newborns and 
5) Babies under 1 year old - Newborns and babies under 1 do not have fully formed microbiomes yet and are often colonized with clostridium difficile without getting an infection. Further, newborns tend to have diarrhea that is mainly caused by other factors (viral infection most commonly)

##### 1) CDI Upon Admission

In [ ]:
# Patients with noted C Diff at admission are already at least suspected of infection
# and for whome risk prediction is not useful
patient_admits["cdi_at_admit"] = ((patient_admits["diagnosis"].str.contains("difficile")) |
                                  (patient_admits["diagnosis"].str.contains("c-diff")) |
                                  (patient_admits["diagnosis"].str.contains("cdiff")) |
                                  (patient_admits["diagnosis"].str.contains("c diff")))

##### 2) No ICU Stay

In [ ]:
# Create binary variable for no ICU stays
patient_admits["no_icu_stay"] = patient_admits["admit_icu_stays"].isnull().map(lambda x: 1 if x == True else 0)

In [ ]:
# Replace null icu stays with 0
patient_admits["admit_icu_stays"].fillna(value=0, inplace=True)

In [ ]:
# Replace null icu LOS with 0
patient_admits["admit_icu_los"].fillna(value=0,inplace=True)

In [ ]:
# Correct for icu_los stored as timedelta and not float
for index in patient_admits.index:
    row_type = type(patient_admits.loc[index,"admit_icu_los"])
    if(row_type != float):
        if((row_type == int)):
            patient_admits.loc[index,"admit_icu_los"] = float(patient_admits.loc[index,"admit_icu_los"])
        else:
            patient_admits.loc[index,"admit_icu_los"] = ((patient_admits.loc[index,"admit_icu_los"].components.days) +
                                                         (patient_admits.loc[index,"admit_icu_los"].components.hours/24) +
                                                         (patient_admits.loc[index,"admit_icu_los"].components.minutes/(60*24)))

In [ ]:
# Change series type to float
patient_admits["admit_icu_los"] = patient_admits["admit_icu_los"].astype(float)

1,190 hospital admissions were without an ICU stay

While these admissions will not be observations for whom C Diff risk is predicted, these admissions may belong to patients who have other ICU stays and for whom this medical history may be important.

##### 3) Diarrhea at Admit

In [ ]:
patient_admits["diarr_at_admit"] = ((patient_admits["diagnosis"].str.contains("loose stool")) |
                                    (patient_admits["diagnosis"].str.contains("diarrhea"))).map(lambda x: 1 if x == True else 0)

##### 4) Newborn Admission

In [ ]:
patient_admits["admission_type"].value_counts()

In [ ]:
patient_admits["newborn_admit"] = patient_admits["admission_type"].map(lambda x: 1 if x == "NEWBORN" else 0)

##### 5) Age Below 1

In [ ]:
patient_admits["age_below_1"] = ((patient_admits["newborn_admit"] == 0) &
                                 (patient_admits["admit_age"] < 1)).map(lambda x: 1 if x == True else 0)

In [ ]:
patient_admits["age_below_1"].sum()

## Export to csv

In [ ]:
patient_admits.to_csv("../Data/1 - Patient Admits.csv")
icu_stays.to_csv("../Data/1 - ICU Stays.csv")